In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from HelperFunctions import *
from PtOccupancyFunctions import *
import scipy.integrate
import pprint

In [ ]:
# path = "/home/helge/Applications/matsim/matsim-bimodal/scenarios/fine_grid/bimodal/output"
path = "/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/CarbonIndex/VaryingEll/SmallLinkRouting/UniformDist/CarSpacing200/OutputUniformDist/200drt"
# path = "/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/CarbonIndex/VaryingEll/SmallLinkRouting/TaxiDataDist2500/output/"

In [ ]:
def getBimDirs(directory, ndrt):
    result = []
    if ndrt:
        ndrt = str(ndrt) + "drt"
        bimdir = os.path.join(directory, ndrt)
    else:
        bimdir = directory
    sub_dirs = []
    sdirs = [
        sdir.path
        for sdir in
            os.scandir(bimdir)
        if sdir.is_dir() and "l_" in sdir.name# and "unimodal" not in sdir.path
    ]

    for sdir in sorted(sdirs, key=lambda x: int(x.split("/")[-1].split("_")[-1])):
        subresult = {}
        subresult["root"] = sdir
        for root, subdirs, files in os.walk(sdir):
            for file in files:
                if file=="0.trips.csv.gz" and "unimodal" not in root:
                    subresult["trips"] = os.path.join(root, file)
                if file=="0.vehicleDistanceStats_drt.csv" and "unimodal" not in root:
                    subresult["drt_dists"] = os.path.join(root, file)
                if file=="trip_success.csv.gz" and "unimodal" not in root:
                    subresult["trip_success"] = os.path.join(root, file)
                if file=="0.CummulativePtDistance.txt" and "unimodal" not in root:
                    subresult["pt_dist"] = os.path.join(root, file)
                if file=="0.drt_occupancy_time_profiles_drt.txt" and "unimodal" not in root:
                    subresult["drt_occupancy"] = os.path.join(root, file)
                if file=="0.occupancyAnalysis.txt" and "unimodal" not in root:
                    subresult["pt_occupancy"] = os.path.join(root, file)
                if file=="0.drt_trips_drt.csv" and "unimodal" not in root:
                    subresult["drt_trips"] = os.path.join(root, file)
                if file=="0.drt_detours_drt.csv" and "unimodal" not in root:
                    subresult["drt_detours"] = os.path.join(root, file)
        result.append(subresult)
    
    return result

def getCarDir(directory):
    result = {}
    result["root"] = getDir(directory, "car")
    for root, subdirs, files in os.walk(result["root"]):
        for file in files:
            if file=="0.trips.csv.gz":
                result["trips"] = os.path.join(root, file)
    
    return result

def getUniDirs(directory, ndrt):
    if ndrt:
        ndrt = str(ndrt) + "drt"
        unidir = os.path.join(directory, ndrt)
    else:
        unidir=directory
    result = {}
    result["root"] = getDir(unidir, "unimodal")
    for root, subdirs, files in os.walk(result["root"]):
        for file in files:
            if file=="0.trips.csv.gz":
                result["trips"] = os.path.join(root, file)
    
    return result

def getDir(path, directory):
    for root, subdirs, files in os.walk(path):
        for subdir in subdirs:
            if subdir == directory:
                return os.path.join(root, subdir)

bim_dirs = getBimDirs(path, None)
uni_dirs = getUniDirs(path, None)
car_dir = getCarDir(path)

In [ ]:
pprint.pprint(bim_dirs)
print('\n---\n')
pprint.pprint(uni_dirs)
print('\n---\n')
pprint.pprint(car_dir)

In [ ]:
def getTrips(paths, mode):
    if mode == "bimodal":
        columns=["person","trav_time","wait_time","traveled_distance","modes"]
    elif mode == "unimodal":
        columns=["person","trav_time","wait_time"]
    elif mode == "car":
        columns=["person", "traveled_distance", "trav_time"]
    path = paths["trips"]
    df = pd.read_csv(path, sep=";").loc[:,columns]
    df["trav_time"] = df["trav_time"].apply(timestmp2sec)
    df.set_index("person", inplace=True)
    if mode=="car":
        df = df.groupby("person").agg(
            {
                "trav_time": np.sum,
                "traveled_distance": np.sum
            }
        )
        return df
    df["wait_time"] = df["wait_time"].apply(timestmp2sec)
    if mode=="unimodal":
        df = df.groupby("person").agg(
            {
                "trav_time": np.sum,
                "wait_time": np.sum,
            }
        )
        return df
    if mode == "bimodal":
        df = df.groupby("person").agg(
            {
                "trav_time": np.sum,
                "wait_time": np.sum,
                "traveled_distance": np.sum,
                "modes": combineModesSeriesStr,
            }
        )
        return df

def getEll(path):
    return path.split("/")[-1].split("_")[-1]

def getDrtVehicleDistances(paths):
    path = paths["drt_dists"]
    df = pd.read_csv(path, sep=";")["drivenDistance_m"]
    return df

def getTripSuccess(paths):
    path = paths["trip_success"]
    df = pd.read_csv(path, sep=";")
    return df

def getCummulativePtDistance(paths):
    path = paths["pt_dist"]
    df = pd.read_csv(path).values[0, 0]
    return df

def getDrtOccupandyAndStandingFrac(paths, exclude_empty_vehicles, count_idle_vehicles=False):
    path = paths["drt_occupancy"]
    df = pd.read_csv(path, sep="\t")
    df["time"] = df["time"].apply(timestmphm2sec)
    drt_occ, drt_deviation = getAverageOcc(
        df.drop(columns="time"), exclude_empty_vehicles=exclude_empty_vehicles, count_idle_vehicles=count_idle_vehicles
    )
    drt_standing_frac = getStandingFraction(df.drop(columns="time"))
    return drt_occ, drt_standing_frac

def getPtOccupancy(paths):
    path = paths["pt_occupancy"]
    av_pt_occ, av_pt_occ_sq, n_pt = getPtOccupancies(
        path, 600
    )
    t_av_pt_occ_av = getAverageTimeSeries(av_pt_occ)
    sigma = np.sqrt(n_pt / (n_pt - 1)) * np.sqrt(
        av_pt_occ_sq - av_pt_occ ** 2
    )
    t_av_pt_occ_sigma = getAverageTimeSeries(sigma)
    return t_av_pt_occ_av, t_av_pt_occ_sigma

def getDrtTrips(paths):
    path = paths["drt_trips"]
    df = pd.read_csv(path, sep=";").loc[:, ["travelDistance_m", "waitTime"]]
    return df

def getDrtDetours(paths):
    path = paths["drt_detours"]
    df = pd.read_csv(path, sep=";").loc[
        :, "distanceDetour"
    ]
    df = df[df < 10]
    return df

# display(getTrips(bim_dirs[0], "bimodal"))

In [ ]:
ell_list = np.empty(12)
cummulative_drt_bimodal = np.empty(12)
cummulative_train_bimodal = np.empty(12)
cummulative_car_bimodal = np.empty(12)
av_drt_occs = np.empty(12)
av_drt_occs_non_empty = np.empty(12)
av_drt_occs_non_standing = np.empty(12)
av_waits = np.empty(12)
serviced_fracts = np.empty(12)
av_pt_occs = np.empty(12)
bi_or_unimodal = np.empty((12,2))
drt_pt_person_km = np.empty((12,2))
av_detours = np.empty(12)
drt_wait_times = np.empty(12)
tt_ratios_beyond_lcut = np.empty(12)
tt_bimodal = np.empty(12)
tt_unimodal = np.empty(12)
standing_fractions = np.empty(12)

In [ ]:
trips_car = getTrips(car_dir, "car")
dists_car = trips_car["traveled_distance"]
cummulative_car = dists_car.sum()
tt_car = trips_car["trav_time"].mean()

trips_unimodal = getTrips(uni_dirs, "unimodal")
tt_unimodal = trips_unimodal["trav_time"].mean()

for i,dic in enumerate(bim_dirs):
    # drt/pt_person_km can be get easier
    ell_list[i] = getEll(dic["root"])
    print("Ell: ", ell_list[i])
    trips = getTrips(dic, "bimodal")
    drt_trips = getDrtTrips(dic)
    drt_veh_dists = getDrtVehicleDistances(dic)
    trip_succ = getTripSuccess(dic)
    av_drt_occs_non_standing[i], standing_frac = getDrtOccupandyAndStandingFrac(dic, False)
    av_drt_occs_non_empty[i], _ = getDrtOccupandyAndStandingFrac(dic, True)
    av_drt_occs[i], _ = getDrtOccupandyAndStandingFrac(dic, False, True)
#     av_drt_occs_non_empty[i] = drt_occupancy_non_empty
#     av_drt_occs_non_standing[i] = drt_occupancy_non_standing
    pt_occupancy, pt_occ_variance = getPtOccupancy(dic)
    drt_detours = getDrtDetours(dic)
    pt_dist = getCummulativePtDistance(dic)
    
    
    drt_pt_person_km[i,0] = drt_trips["travelDistance_m"].sum()
    drt_pt_person_km[i,1] = trips["traveled_distance"].sum()-drt_pt_person_km[i,0]
    serviced_fracts[i] = len(trip_succ[trip_succ["tripSuccess"]]) / len(trip_succ)
    print("Servability: ", serviced_fracts[i])
    dists_car_for_rejected = dists_car.loc[
        trip_succ[~trip_succ["tripSuccess"]]["personId"].to_numpy()
    ]
    cummulative_drt_bimodal[i] = drt_veh_dists.sum()
    cummulative_car_bimodal[i] = dists_car_for_rejected.sum()
    cummulative_train_bimodal[i] = pt_dist
    standing_fractions[i] = standing_frac
    av_waits[i] = trips["wait_time"].mean()
    av_pt_occs[i] = pt_occupancy
#     av_trav_time_bimodal = trav_times[pt_in_trips]
    pt_in_trips = trips["modes"].str.contains("pt")
    count_pt_in_trips = pt_in_trips.value_counts()
    bi_or_unimodal[i,0] = count_pt_in_trips[False]
    bi_or_unimodal[i,1] = count_pt_in_trips[True]
    av_detours[i] = drt_detours.mean()
    tt_bimodal[i] = trips["trav_time"].mean()
    drt_wait_times[i] = trips["wait_time"].mean()
    idx_pt_in_trips = pt_in_trips[pt_in_trips].index
    tt_ratios_beyond_lcut[i] = trips["trav_time"][idx_pt_in_trips].sum()/trips_car["trav_time"][idx_pt_in_trips].sum()
    print("-----")

bi_or_unimodal = pd.DataFrame(bi_or_unimodal, index=ell_list, columns=["Unimodal","Bimodal"])
drt_pt_person_km = pd.DataFrame(drt_pt_person_km, index=ell_list, columns=["DRT","PT"])

---
## Theoretical prediction

In [ ]:
nu = 1 # per day
L = 1 #10km
E = 100000 #per 100km^2
R0 = 800/130
mu = 96 # per day
#nuELsquared = 1e5/(24*3600)
# delta = 1.5
# b = 2
zeta = 1
# U = 35
beta = 1/6*(np.sqrt(2)+np.log(1+np.sqrt(2)))
# f = 0.85
D = 0.25
v_drt = 30*24/10
B = 300/(L*L) # = 300

x = ell_list/(10000) # Factor 10000, because units in simulation are meters
# ell_list = np.linspace(ell_min,ell_max,M)
#f = np.linspace(0,1,N) use f from simulations
f = serviced_fracts
capacity = np.zeros_like(x)

def delta_av_theory(delta_max, b, mode='mft'):
    delta_max = 1.5
    delta_bar = 2 * delta_max / 3 + 1 / (3 * delta_max)
    if mode=='mft':
        return np.exp(np.log(delta_bar) / np.log(2) * np.log((2*b - 1)))
    elif mode=='bimodal':
        return np.exp(np.log(delta_bar) / np.log(2) * np.log((b + 1) / 2))

def uniform_dist(x):
    if (x>L/2):
        return 0
    else:
        return 2/L
    
normalize = scipy.integrate.quad(uniform_dist,0,np.infty)[0]
assert np.isclose(1, normalize), "Should already be normalized"

def cummulative_prob(x):
    return scipy.integrate.quad(uniform_dist,0,x)[0]/normalize

def size(x,dx):
    return uniform_dist(x)*np.arccos(dx/x)/(2*np.pi*normalize)

def tramsize(ell, mode='int'):
    scl = int(L/ell)
#    print(scl)
    sm = 0
    if mode=='int':
        for i in range(0,100):
            dx = ell*i
            sm = sm + 2*scipy.integrate.quad(size,np.maximum(i*ell,zeta*ell),np.infty,args=(dx))[0]
        sm = sm - (1-cummulative_prob(zeta*ell))/4
        sm = sm*nu*E*ell*ell
    elif mode=='av':
        norm = scipy.integrate.quad(uniform_dist,zeta*ell,np.infty)[0]
        if not np.isclose(zeta*ell,L/2):
            average_pt_dist = scipy.integrate.quad(lambda x: x*uniform_dist(x)/norm,zeta*ell,np.infty)[0]
        else:
            average_pt_dist = L/2
#         print("av pt dist:", average_pt_dist)
        sm = average_pt_dist/np.pi*E*nu*ell * (1-cummulative_prob(zeta*ell))
    
    sm /= mu
    return sm

def rpr(x):
    if x==0:
        return 0
    else:
        return x*uniform_dist(x)/normalize
    
def getCarbonIndex(ell, f, b, delta, R0):
    den = nu*E*L*L*scipy.integrate.quad(rpr,0,np.infty)[0]
    Ci = np.zeros_like(ell)
    Ci_drt = np.zeros_like(ell)
    Ci_pt = np.zeros_like(ell)
    Ci_car = np.zeros_like(ell)

    for i in range(len(ell)):
        dbus = ((scipy.integrate.quad(rpr,0,zeta*ell[i]))[0] + 2*beta*ell[i]*(1-cummulative_prob(zeta*ell[i])))*nu*E*L*L
        #mu = tramsize(l[i])/100
        cap = tramsize(ell[i])
        ctram = 4*mu*L*np.ceil(L/ell[i])#+1)
#         ctram = 4*mu*L*L/ell[i]
#         print(ctram*10000) # Distance is measured in 10km
        ctram *= R0
        Ci_drt[i] = ((delta[i]/b[i])*f[i])*dbus/den
        Ci_pt[i] = ctram/den
        Ci_car[i] = (1-f[i])*scipy.integrate.quad(rpr,0,np.infty)[0]*nu*E*L*L/den
        Ci[i] = Ci_drt[i] + Ci_pt[i] + Ci_car[i]
#         Ci[i] = (((delta[i]/b[i])*f[i])*dbus+(1-f[i])*scipy.integrate.quad(rpr,0,np.infty)[0]*nu*E*L*L + ctram)/den
#         capacity[i] = cap*f[i]
        
    return Ci, Ci_pt, Ci_drt, Ci_car

def getTramsizes(ells, f):
    result = np.empty((len(ells),2))
    for i,ell in enumerate(ells):
        result[i,0] = tramsize(ell)*f[i]
        result[i,1] = tramsize(ell, mode='av')*f[i]
    
    return result

def getDrtNonEmptyOccupancy(ells, v_drt, B, detours):
#     print(ells, v_drt, B, detours)
    result = np.empty_like(ells)
    prefactor = nu*E/(v_drt*B)
#     print(prefactor)
    
    for i, ell in enumerate(ells):
        a = scipy.integrate.quad(rpr,0,zeta*ell)[0]
        b = scipy.integrate.quad(uniform_dist, zeta*ell, np.infty)[0]
#         print(ell, a, b)
        result[i] = prefactor*detours[i]*(a+2*beta*ell*b)
        
    return result

delta_av_theory = delta_av_theory(1.5,av_drt_occs_non_empty, mode='mft')

Ci_theory_mft, Ci_theory_pt_mft, Ci_theory_drt_mft, Ci_theory_car_mft = \
    getCarbonIndex(x, f, av_drt_occs_non_standing, delta_av_theory, R0)

Ci_theory, Ci_theory_pt, Ci_theory_drt, Ci_theory_car = \
    getCarbonIndex(x, f, av_drt_occs_non_standing, av_detours, R0)

tramsizes = getTramsizes(x, f)#[1 for _ in range(len(x))])

occupancy_theory = getDrtNonEmptyOccupancy(x, v_drt, 300, av_detours)

---
# Visualization

In [ ]:
%matplotlib notebook
save_figs = False

fig, ax = plt.subplots()
xlabel = r"$\ell/D$"
x_rescaled = x/D
# Ci_old = Ci

Ci_data_drt = cummulative_drt_bimodal/cummulative_car
Ci_data_car = cummulative_car_bimodal/cummulative_car
Ci_data_pt = R0*cummulative_train_bimodal/cummulative_car
Ci_data =  Ci_data_drt + Ci_data_car + Ci_data_pt

ax.plot(x_rescaled, Ci_data, 'o-', label="Simulation (R0 ≈ {})".format(round(R0,1)), color='#1462e0')
# ax.plot(x_rescaled, Ci_data_drt, 'o-', color='orange', label="Simulation drt contribution")
# ax.plot(x_rescaled, Ci_data_pt, 'o-', color='green', label="Simulation pt contribution")
# ax.plot(x_rescaled, Ci_data_car, 'o-', color='coral', label="Simulation car contribution")

#     print(np.argmin(Ci_data))
print("Min of C_I with R0=", round(R0,2), "at: ", x[np.argmin(Ci_data)], "value: ", np.min(Ci_data))

ax.plot(x_rescaled, Ci_theory, "o--", c='r', label="Theory (R0 ≈ {})".format(round(R0,1)))
# ax.plot(ell_list, Ci_theory_drt, "o--", c='orange', label="Theory drt contribution")
# ax.plot(ell_list, Ci_theory_pt, "o--", c='green', label="Theory pt contribution")
# ax.plot(ell_list, Ci_theory_car, "o--", c='coral', label="Theory car contribution")
ax.set_ylabel(r"$C_I$")
# print(Ci_data/Ci)
# print(ell_list[np.argmin(Ci)])
ax.set_xlabel(xlabel)
ax.legend()

ax.set_ylim((0,None))
print(Ci_data)

if save_figs:
    fig.savefig("CarbonIdxVaryingl.png", dpi=200)

In [ ]:
fig, ax = plt.subplots()

cmap = plt.get_cmap('viridis')

Ci_data_drt = cummulative_drt_bimodal/cummulative_car
Ci_data_car = cummulative_car_bimodal/cummulative_car
for R0 in [1, 4, 7]:
    Ci_data_pt = R0*cummulative_train_bimodal/cummulative_car
    Ci_data = Ci_data_drt+Ci_data_car+Ci_data_pt
    Ci_theory_mft, Ci_theory_pt, Ci_theory_drt, Ci_theory_car = getCarbonIndex(
        x, f, av_drt_occs_non_standing, delta_av_theory, R0
    )
    Ci_theory, _, _, _ = getCarbonIndex(
        x, f, av_drt_occs_non_standing, av_detours, R0
    )
    Ci_data = (
        cummulative_car_bimodal
        + cummulative_drt_bimodal
        + R0 * cummulative_train_bimodal
    ) / cummulative_car
    ax.plot(x_rescaled, Ci_data, "o-", label="Simulation"+r"($\epsilon_0$ ≈ {})".format(round(R0, 1)), color=cmap(R0/7))
    ax.plot(x_rescaled, Ci_theory, "o--", label="Theory"+r"($\epsilon_0$ ≈ {})".format(round(R0,1)), color=cmap(R0/7))
#     ax.plot(x_rescaled, Ci_theory_mft, "o", ls='dotted',label="Theory (R0 ≈ {})".format(round(R0,1)), color=cmap(R0/7))
#     ax.axvline(x_rescaled[np.nanargmin(Ci_data)], color=cmap(R0/7), ls=(0, (1, 1)))
#     ax.axvline(x_rescaled[np.nanargmin(Ci_theory)], color=cmap(R0/7), ls=(0, (3, 10, 1, 10)))
#     print("Simulation min of C_I with R0=", round(R0,2), "at: ", x_rescaled[np.argmin(Ci_data)])
#     print("Theory min of C_I with R0=", round(R0,2), "at: ", x_rescaled[np.argmin(Ci_theory)])

ax.set_xlabel(xlabel)
ax.grid()
ax.legend()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_ylabel(r"$C_I$")

ax.set_ylim((0,None))
print(Ci_data)

# save_figs=False
if save_figs:
#     fig.savefig("CarbonIdxVaryinglAndR0SimTheo.png", dpi=200)
    fig.savefig("CarbonIdxVaryinglAndR0SimTheo.pdf")

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

# bi_or_unimodal.set_index(round(1/bi_or_unimodal.reset_index()["index"]), inplace=True)
bi_or_unimodal.set_index(np.round(x_rescaled, decimals=2), inplace=True)
bi_or_unimodal.plot.bar(ax = ax, xlabel=xlabel, ylabel="frequency")

ax.set_title("Travel Mode Share")

if save_figs:
    fig.savefig("BiOrUnimodalBarPlot.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots(constrained_layout=True)

# bi_or_unimodal.set_index(round(1/bi_or_unimodal.reset_index()["index"]), inplace=True)
drt_pt_person_km.set_index(np.round(x_rescaled, decimals=2), inplace=True)
drt_pt_person_km.plot.bar(ax = ax, xlabel=xlabel, ylabel="meter travelled")

patches, labels = ax.get_legend_handles_labels()

ax.legend(patches, labels)#, frameon=False, bbox_to_anchor=(1.05, 1))
# display(drt_pt_person_km)

ax.set_title("Meters Travelled")

# save_figs = True
if save_figs:
    fig.savefig("PersonKmPerTravelMode.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, serviced_fracts, 'o-')
ax.set_ylabel("f")
ax.set_xlabel(xlabel)

ax.set_ylim((0,1.1))

if save_figs:
    fig.savefig("ServicedFraction.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

def getWaitTimes(mu_s, link_tt, bi_uni_share, p_drt):
    notp_d = 1-p_d
    ptwait = 1/mu_s
    drtwait = link_tt*sum([i*notp_d**(2*i*(i+1)) for i in range(1,100)])
    twaits = bi_uni_share['Bimodal'] * (ptwait + 2*drtwait) + bi_uni_share['Unimodal'] * drtwait
    return twaits, drtwait, ptwait
    
bi_uni_share = bi_or_unimodal.divide(bi_or_unimodal.sum(axis=1), axis=0)
mu_s = mu/(24*3600)
link_tt = 100/(30/3.6)
p_d = 300/10000
totalwait, drtwait, ptwait = getWaitTimes(mu_s, link_tt, bi_uni_share, p_d)


ax.plot(x_rescaled, av_waits, 'o-')
# ax.plot(x_rescaled, totalwait, 'o--')
ax.set_ylabel("Average waiting time (s)")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("AverageWaitingTime.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

# test = (av_drt_occs - av_drt_occs.min()) /(av_drt_occs.max() - av_drt_occs.min())
# print(test)
ax.plot(x_rescaled, drt_wait_times, '-o', label='Waiting Time Drt')
ax.plot(x_rescaled, [drtwait for _ in x], '--o')
ax.set_ylabel("t")
ax.set_xlabel(xlabel)
ax.legend()

plt.show()

In [ ]:
fig, ax = plt.subplots()
#TODO fix this

ax.plot(x_rescaled, av_drt_occs, 'o-')
ax.plot(x_rescaled, occupancy_theory, 'o--')
ax.set_ylabel(r"$\langle b_\mathrm{drt}\rangle$")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("AverageOccupancy.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()
#TODO fix this

ax.plot(x_rescaled, av_drt_occs_non_empty, 'o-')
ax.set_ylabel(r"$\langle b_\mathrm{drt}^*\rangle$")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("AverageNonEmptyOccupancy.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, av_drt_occs_non_standing, 'o-')
ax.set_ylabel(r"$\langle b_\mathrm{drt}\rangle$")
ax.set_xlabel(xlabel)

print(save_figs)
if save_figs:
    fig.savefig("AverageOccupancy.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, av_pt_occs, 'o-', label="Simulation")

# ax.plot(ell_list, tramsizes[:,1], "o--", c='r', label="Theory computed by"+r" $\langle n\rangle$")
# ax.plot(ell_list, tramsizes[:,0], "o--", c='g', label="Theory computed by"+r" $\tilde{I}$")
ax.plot(x_rescaled, tramsizes[:,1], "o--", c='g', label="Theory")
ax.legend()

ax.set_ylabel(r"$\langle b_\mathrm{pt}\rangle$")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("AveragePtOccupancy.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, av_detours, 'o-')
ax.set_ylabel(r"$\langle \delta\rangle$")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("AverageDrtDetour.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, av_drt_occs_non_empty/av_detours, 'o-')
ax.set_ylabel(r"$\langle b\rangle/\langle\delta\rangle$")
ax.set_xlabel(xlabel)

if save_figs:
    fig.savefig("BOverDelta.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, tt_bimodal, 'o-', label='bimodal')
ax.plot(x_rescaled, tt_car, 'o-', label='car')
ax.plot(x_rescaled, tt_unimodal, 'o-', label='unimodal')
ax.set_ylabel(r"$t$")
ax.set_xlabel(xlabel)
ax.legend()

if save_figs:
    fig.savefig("AverageTravelTimes.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, tt_ratios_beyond_lcut, 'o-', label=r"$TT_\mathrm{bimodal}\,/\,TT_\mathrm{car}$")
ax.set_xlabel(xlabel)
ax.legend()

if save_figs:
    fig.savefig("TTRatiosBeyondLCut.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(x_rescaled, standing_fractions, 'o-', label="Standing Fraction of DRT vehicles")
ax.set_xlabel(xlabel)
ax.legend()

if save_figs:
    fig.savefig("StandingDrtFraction.png", dpi=200)
plt.show()

# Debug

In [ ]:
plt.figure()
plt.scatter(ell_list, tramsizes[:,0], label='int')
plt.scatter(ell_list, tramsizes[:,1], label='av')
plt.legend()

plt.show()